In [1]:
import numpy as np 
import torch 
import torch.nn as nn
from  Data_Loader import DATA_1M
from helper_functions import accuracy_fn
from CnnModel import NeuralNetCNN
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, recall_score)

c:\Users\janathan.pena\Documents\GitHub\Deep-learning-AICT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
class Count:

    def __init__(self,mymin,mymax):
        self.mymin=mymin
        self.mymax=mymax
        self.current=None
   
    def __getattribute__(self, item):
        if item.startswith('cur'):
            return f"gettinh {object.__getattribute__(self,item)}"
        return object.__getattribute__(self,item) 
        # or you can use ---return super().__getattribute__(item)

obj1 = Count(1,10)
print(obj1.mymin)
print(obj1.mymax)
print(obj1.current)

1
10


AttributeError: 

### Com Fourier

In [2]:
%%time
data = DATA_1M(seconds=40,columns=20000, jump_time =0, n_jumps=1)
print(data)
data_fourier = data(Fourier=False)
data_fourier.shape

Numero de linhas 100000000 equivalente a 40 segundos
 Pulando em 0 segundos , em 1 vezes
Tamanho da memoria ocupada :4577.87 MB
CPU times: total: 18min 31s
Wall time: 25min 16s


(30000, 20001)

In [3]:
%%time
Torch = NeuralNetCNN(columns= data_fourier.shape[1] -1)

CPU times: total: 156 ms
Wall time: 265 ms


In [4]:
Torch.Cnn

CNNModel(
  (layer1): Sequential(
    (0): Conv1d(20000, 512, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(512, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Conv1d(256, 256, kernel_size=(2,), stride=(1,), padding=(1,))
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv1d(256, 128, kernel

In [5]:
data.Spliting(data= data_fourier, random_state= 38, test_size = 0.275, shuffle = True, inplace= False)


X_train shape: (21750, 20000) float64
X_Test shape: (8250, 20000) float64
y_train shape: (21750,) float64
y_test shape: (8250,) float64

--------
Valor 0: 10000 ocorrência(s)- 0.33%
Valor 1: 10000 ocorrência(s)- 0.33%
Valor 2: 10000 ocorrência(s)- 0.33%
Dataset :  (30000, 20001)


In [7]:
train_dataloader , test_dataloader = data.DataLoaders(batch_size=64, inplace=True)

X shape , Y shape torch.Size([64, 20000]) torch.Size([64])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x000001E0EE51BDD0>, <torch.utils.data.dataloader.DataLoader object at 0x000001E0EE5195D0>)
Length of train dataloader: 340 batches of 64
Length of test dataloader: 129 batches of 64


In [8]:
%%time
Torch.training_loop(data_loader_train=train_dataloader,
data_loader_test = test_dataloader,
        model=Torch.Cnn, 
        loss_fn=Torch.loss_fn,
        optimizer=Torch.optimizer,
        accuracy_fn=accuracy_fn,
        device=Torch.device,
        epochs = 15)



Training on cpu


  0%|          | 0/15 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.22600 | Train accuracy: 54.94%


  7%|▋         | 1/15 [14:07<3:17:51, 847.99s/it]

Test loss: 70.34335 | Test accuracy: 72.41%
 Epoch: 1
---------
Train loss: 1.00819 | Train accuracy: 81.49%


...

Test loss: 65.69078 | Test accuracy: 73.42%
 Epoch: 13
---------
Train loss: 0.77818 | Train accuracy: 96.79%


 93%|█████████▎| 14/15 [3:06:00<13:08, 788.58s/it]

Test loss: 66.02454 | Test accuracy: 73.10%
 Epoch: 14
---------
Train loss: 0.77613 | Train accuracy: 96.98%


100%|██████████| 15/15 [3:19:15<00:00, 797.02s/it]

Test loss: 66.37802 | Test accuracy: 72.58%
CPU times: total: 19h 47min 1s
Wall time: 3h 19min 15s


In [9]:
print(Torch(test= True, train= True))


    Test Accuracy  Test Loss   Train Accuracy  Train Loss
0       72.581696   66.378017       54.935321    1.225999
1       72.581696   66.378017       81.494247    1.008194
2       72.581696   66.378017       93.298271    0.853632
3       72.581696   66.378017       96.373230    0.801400
4       72.581696   66.378017       96.594669    0.790143
5       72.581696   66.378017       96.612200    0.786399
6       72.581696   66.378017       96.501906    0.785633
7       72.581696   66.378017       96.021412    0.788205
8       72.581696   66.378017       95.876055    0.789605
9       72.581696   66.378017       95.980052    0.787655
10      72.581696   66.378017       96.180215    0.785071
11      72.581696   66.378017       96.398250    0.782625
12      72.581696   66.378017       96.510246    0.781021
13      72.581696   66.378017       96.785982    0.778176
14      72.581696   66.378017       96.978145    0.776134


In [10]:
class_names = ['CLEAR','WIFI','LTE']

print(classification_report(data.y_test, Torch.Making_Predictions(model = Torch.Cnn, data_loader= test_dataloader),target_names=class_names))

              precision    recall  f1-score   support

       CLEAR       0.55      0.86      0.67      2716
        WIFI       0.71      0.33      0.45      2809
         LTE       1.00      1.00      1.00      2725

    accuracy                           0.73      8250
   macro avg       0.75      0.73      0.71      8250
weighted avg       0.75      0.73      0.71      8250



### Sem Fourier

In [11]:
%%time
data = DATA_1M(seconds=40,columns=20000, jump_time =0, n_jumps=1)
print(data)
data_fourier = data(Fourier=True)
data_fourier.shape

Numero de linhas 100000000 equivalente a 40 segundos
 Pulando em 0 segundos , em 1 vezes
Tamanho da memoria ocupada :4577.87 MB
CPU times: total: 14min 1s
Wall time: 34min 29s


(30000, 20001)

In [12]:
data.Spliting(data= data_fourier, random_state= 38, test_size = 0.275, shuffle = True, inplace= False)


X_train shape: (21750, 20000) float64
X_Test shape: (8250, 20000) float64
y_train shape: (21750,) float64
y_test shape: (8250,) float64

--------
Valor 0: 10000 ocorrência(s)- 0.33%
Valor 1: 10000 ocorrência(s)- 0.33%
Valor 2: 10000 ocorrência(s)- 0.33%
Dataset :  (30000, 20001)


In [13]:
train_dataloader , test_dataloader = data.DataLoaders(batch_size=64, inplace=True)

X shape , Y shape torch.Size([64, 20000]) torch.Size([64])
----------------

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x000001E0EE4EFFD0>, <torch.utils.data.dataloader.DataLoader object at 0x000001E0EE1D94D0>)
Length of train dataloader: 340 batches of 64
Length of test dataloader: 129 batches of 64


In [14]:
%%time
Torch.training_loop(data_loader_train=train_dataloader,
data_loader_test = test_dataloader,
        model=Torch.Cnn, 
        loss_fn=Torch.loss_fn,
        optimizer=Torch.optimizer,
        accuracy_fn=accuracy_fn,
        device=Torch.device,
        epochs = 15)



Training on cpu


  0%|          | 0/15 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 1.34507 | Train accuracy: 38.98%


  7%|▋         | 1/15 [13:47<3:13:01, 827.28s/it]

Test loss: 84.69437 | Test accuracy: 41.67%
 Epoch: 1
---------
Train loss: 1.23940 | Train accuracy: 48.65%


...

Test loss: 87.00118 | Test accuracy: 39.11%
 Epoch: 13
---------
Train loss: 0.79937 | Train accuracy: 94.61%


 93%|█████████▎| 14/15 [3:10:39<13:34, 814.03s/it]

Test loss: 86.97332 | Test accuracy: 39.22%
 Epoch: 14
---------
Train loss: 0.79809 | Train accuracy: 94.72%


100%|██████████| 15/15 [3:24:08<00:00, 816.55s/it]

Test loss: 87.60427 | Test accuracy: 38.22%
CPU times: total: 20h 14min 1s
Wall time: 3h 24min 8s


In [15]:
print(Torch(test= True, train= True))





    Test Accuracy  Test Loss   Train Accuracy  Train Loss
0       38.215467   87.604266       38.984375    1.345068
1       38.215467   87.604266       48.654684    1.239400
2       38.215467   87.604266       65.315564    1.084490
3       38.215467   87.604266       74.356447    0.999285
4       38.215467   87.604266       79.091605    0.954565
5       38.215467   87.604266       82.588848    0.920301
6       38.215467   87.604266       85.979541    0.885640
7       38.215467   87.604266       89.214495    0.854497
8       38.215467   87.604266       90.866353    0.837193
9       38.215467   87.604266       92.227158    0.823987
10      38.215467   87.604266       92.979133    0.815760
11      38.215467   87.604266       93.506774    0.810251
12      38.215467   87.604266       94.273046    0.802804
13      38.215467   87.604266       94.605971    0.799371
14      38.215467   87.604266       94.717967    0.798092


In [16]:
class_names = ['CLEAR','WIFI','LTE']

print(classification_report(data.y_test, Torch.Making_Predictions(model = Torch.Cnn, data_loader= test_dataloader),target_names=class_names))

              precision    recall  f1-score   support

       CLEAR       0.39      0.55      0.45      2716
        WIFI       0.41      0.34      0.37      2809
         LTE       0.35      0.25      0.29      2725

    accuracy                           0.38      8250
   macro avg       0.38      0.38      0.37      8250
weighted avg       0.38      0.38      0.37      8250

